In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import (
    RandomOverSampler,
    SMOTE,
    ADASYN,
    BorderlineSMOTE,
    SVMSMOTE
)

In [ ]:
# kagglehub.dataset_download("shivamb/machine-predictive-maintenance-classification")

maintenance = pd.read_csv("C:\\Users\\danis\\Downloads\\Datasets smote\\predictive_maintenance.csv")

In [3]:
maintenance

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [4]:
del maintenance['UDI']
del maintenance['Product ID']
del maintenance['Type']
del maintenance['Failure Type']

In [5]:
maintenance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Air temperature [K]      10000 non-null  float64
 1   Process temperature [K]  10000 non-null  float64
 2   Rotational speed [rpm]   10000 non-null  int64  
 3   Torque [Nm]              10000 non-null  float64
 4   Tool wear [min]          10000 non-null  int64  
 5   Target                   10000 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 468.9 KB


In [6]:
from sklearn.model_selection import train_test_split
from collections import Counter

In [7]:
maintenance['Target'].value_counts()

Target
0    9661
1     339
Name: count, dtype: int64

In [8]:
df_resampled, y_resampled = maintenance.copy(), maintenance['Target']
df_resampled2 = df_resampled.copy()     # saving same data to be used for our proposed method 
print("Original class distribution:", Counter(y_resampled))
del df_resampled['Target']

Original class distribution: Counter({0: 9661, 1: 339})


In [9]:
def evaluate_oversampling(X, y, sampler, classifier):

    # Train-test split on resampled data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)
    print("Class distribution after SMOTE:", Counter(y_train_resampled))

    
    # Train the classifier
    classifier.fit(X_train_resampled, y_train_resampled)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1, X_test, y_test

oversamplers = {
    "RandomOverSampler": RandomOverSampler(sampling_strategy='minority'),
    "SMOTE": SMOTE(sampling_strategy='minority'),
    "ADASYN": ADASYN(sampling_strategy='minority'),
    "BorderlineSMOTE": BorderlineSMOTE(sampling_strategy='minority'),
    "SVMSMOTE": SVMSMOTE(sampling_strategy='minority')
}
resultsNB = []
for name, sampler in oversamplers.items():
    accuracy, precision, recall, f1, xtest, ytest = evaluate_oversampling(df_resampled, y_resampled, sampler=sampler, classifier=GaussianNB())
    resultsNB.append({"Method": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

resultsNB = pd.DataFrame(resultsNB)

# Print results
print(resultsNB)

Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({1: 7745, 0: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
              Method  Accuracy  Precision  Recall  F1 Score
0  RandomOverSampler    0.8440   0.964175  0.8440  0.892258
1              SMOTE    0.8380   0.962688  0.8380  0.888400
2             ADASYN    0.7920   0.965279  0.7920  0.859620
3    BorderlineSMOTE    0.7955   0.962703  0.7955  0.861740
4           SVMSMOTE    0.8885   0.959220  0.8885  0.918337


In [10]:
resultsKNN = []
for name, sampler in oversamplers.items():
    accuracy, precision, recall, f1, xtest, ytest = evaluate_oversampling(df_resampled, y_resampled, sampler=sampler, classifier=KNeighborsClassifier())
    resultsKNN.append({"Method": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

resultsKNN = pd.DataFrame(resultsKNN)

# Print results
print(resultsKNN)

Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({1: 7745, 0: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
              Method  Accuracy  Precision  Recall  F1 Score
0  RandomOverSampler    0.9300   0.956112  0.9300  0.941681
1              SMOTE    0.8905   0.962449  0.8905  0.920173
2             ADASYN    0.8790   0.961217  0.8790  0.913069
3    BorderlineSMOTE    0.9170   0.956774  0.9170  0.934410
4           SVMSMOTE    0.9255   0.956281  0.9255  0.939159


In [11]:
resultsRF = []
for name, sampler in oversamplers.items():
    accuracy, precision, recall, f1, xtest, ytest = evaluate_oversampling(df_resampled, y_resampled, sampler=sampler, classifier=RandomForestClassifier())
    resultsRF.append({"Method": name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

resultsRF = pd.DataFrame(resultsRF)

# Print results
print(resultsRF)

Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({1: 7745, 0: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
Class distribution after SMOTE: Counter({0: 7722, 1: 7722})
              Method  Accuracy  Precision  Recall  F1 Score
0  RandomOverSampler    0.9830   0.981597  0.9830  0.981929
1              SMOTE    0.9580   0.972865  0.9580  0.963830
2             ADASYN    0.9555   0.971751  0.9555  0.961930
3    BorderlineSMOTE    0.9685   0.974771  0.9685  0.971076
4           SVMSMOTE    0.9730   0.977479  0.9730  0.974831


_______________________________________________________________________________________________________________________________________________________________________

In [12]:
maintenance[maintenance['Target']==1].corr()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
Air temperature [K],1.000000,0.848669,-0.142306,0.035488,-0.229465,NaN
Process temperature [K],0.848669,1.000000,-0.074655,0.016210,-0.104193,NaN
Rotational speed [rpm],-0.142306,-0.074655,1.000000,-0.879944,-0.108680,NaN
Torque [Nm],0.035488,0.016210,-0.879944,1.000000,-0.021317,NaN
Tool wear [min],-0.229465,-0.104193,-0.108680,-0.021317,1.000000,NaN
Target,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
maintenance[maintenance['Target']==1].corr().abs().sum().sort_values()

Target                     0.000000
Tool wear [min]            1.463655
Torque [Nm]                1.952960
Process temperature [K]    2.043727
Rotational speed [rpm]     2.205584
Air temperature [K]        2.255928
dtype: float64

In [34]:
maintenance[maintenance['Target']==1].count()

Air temperature [K]        339
Process temperature [K]    339
Rotational speed [rpm]     339
Torque [Nm]                339
Tool wear [min]            339
Target                     339
dtype: int64

In [35]:
7722-339

7383

In [36]:
minority_samples = maintenance[maintenance['Target']==1]
random_values = minority_samples['Air temperature [K]'].sample(n=7383, replace=True)
random_values = random_values.reset_index()
random_values = pd.DataFrame(random_values)
del random_values['index']
random_values

,Air temperature [K]
0,300.3
1,296.9
2,302.6
3,302.5
4,300.0
...,...
7378,297.4
7379,297.2
7380,302.6
7381,300.3


In [19]:
maintenance.columns

Index(['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Target'],
      dtype='object')

In [37]:
random_values['Process temperature [K]'] = np.nan
random_values['Rotational speed [rpm]'] = np.nan
random_values['Torque [Nm]'] = np.nan
random_values['Tool wear [min]'] = np.nan
random_values['Target'] = 1

In [38]:
random_values

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
0,300.3,NaN,NaN,NaN,NaN,1
1,296.9,NaN,NaN,NaN,NaN,1
2,302.6,NaN,NaN,NaN,NaN,1
3,302.5,NaN,NaN,NaN,NaN,1
4,300.0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...
7378,297.4,NaN,NaN,NaN,NaN,1
7379,297.2,NaN,NaN,NaN,NaN,1
7380,302.6,NaN,NaN,NaN,NaN,1
7381,300.3,NaN,NaN,NaN,NaN,1


In [22]:
def calculate_percentiles(nums):
    indexed_nums = [(num, i) for i, num in enumerate(nums)]
    sorted_nums = []
    for num_index in indexed_nums:
        inserted = False
        for i, sorted_num_index in enumerate(sorted_nums):
            if num_index[0] < sorted_num_index[0]:
                sorted_nums.insert(i, num_index)
                inserted = True
                break
        if not inserted:
            sorted_nums.append(num_index)
    length = len(sorted_nums)
    percentiles = [0] * length
    for i, num_index in enumerate(sorted_nums):
        original_index = num_index[1]
        percentile = ((i + 1) / length) * 100
        percentiles[original_index] = percentile
    return percentiles

def dataframe_to_percentiles(df):
    df_percentiles = df.copy()
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df_percentiles[column] = calculate_percentiles(df[column].tolist())
    return df_percentiles


In [39]:
random_values['Air temperature [K]'] = calculate_percentiles(random_values['Air temperature [K]'])
random_values['Air temperature [K]'] = 'P' + random_values['Air temperature [K]'].round().astype(str,errors='ignore')

In [40]:
random_values['Air temperature [K]'].unique()

array(['P32.0', 'P3.0', 'P75.0', 'P73.0', 'P31.0', 'P57.0', 'P42.0',
       'P38.0', 'P48.0', 'P66.0', 'P27.0', 'P7.0', 'P14.0', 'P54.0',
       'P95.0', 'P16.0', 'P79.0', 'P47.0', 'P28.0', 'P11.0', 'P19.0',
       'P23.0', 'P18.0', 'P94.0', 'P39.0', 'P24.0', 'P9.0', 'P86.0',
       'P0.0', 'P82.0', 'P60.0', 'P64.0', 'P8.0', 'P2.0', 'P61.0',
       'P90.0', 'P30.0', 'P69.0', 'P6.0', 'P99.0', 'P49.0', 'P35.0',
       'P21.0', 'P15.0', 'P20.0', 'P50.0', 'P52.0', 'P97.0', 'P83.0',
       'P44.0', 'P88.0', 'P45.0', 'P1.0', 'P10.0', 'P98.0', 'P26.0',
       'P29.0', 'P33.0', 'P84.0', 'P5.0', 'P70.0', 'P87.0', 'P13.0',
       'P96.0', 'P100.0', 'P91.0', 'P89.0', 'P40.0', 'P46.0', 'P76.0',
       'P58.0', 'P4.0', 'P74.0', 'P34.0', 'P22.0', 'P53.0', 'P80.0',
       'P67.0', 'P36.0', 'P55.0', 'P12.0', 'P51.0', 'P71.0', 'P62.0',
       'P92.0', 'P25.0', 'P65.0', 'P77.0', 'P59.0', 'P43.0', 'P81.0',
       'P85.0', 'P17.0', 'P68.0', 'P72.0', 'P56.0', 'P41.0', 'P37.0',
       'P93.0', 'P63.0', 'P78

In [29]:
maintenanceP = dataframe_to_percentiles(minority_samples)
maintenanceP = 'P' + maintenanceP.round().astype(str,errors='ignore')
maintenanceP['Target'] = 1

In [31]:
maintenanceP

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
50,P22.0,P21.0,P99.0,P1.0,P43.0,1
69,P22.0,P20.0,P72.0,P87.0,P59.0,1
77,P21.0,P19.0,P80.0,P19.0,P75.0,1
160,P16.0,P9.0,P9.0,P72.0,P86.0,1
161,P15.0,P6.0,P73.0,P45.0,P88.0,1
...,...,...,...,...,...,...
9758,P19.0,P30.0,P91.0,P9.0,P90.0,1
9764,P19.0,P25.0,P12.0,P88.0,P5.0,1
9822,P19.0,P24.0,P45.0,P73.0,P57.0,1
9830,P16.0,P23.0,P33.0,P59.0,P72.0,1


In [42]:
ZERO = pd.concat([maintenanceP,random_values], ignore_index=True)
ZERO

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
0,P22.0,P21.0,P99.0,P1.0,P43.0,1
1,P22.0,P20.0,P72.0,P87.0,P59.0,1
2,P21.0,P19.0,P80.0,P19.0,P75.0,1
3,P16.0,P9.0,P9.0,P72.0,P86.0,1
4,P15.0,P6.0,P73.0,P45.0,P88.0,1
...,...,...,...,...,...,...
7717,P8.0,NaN,NaN,NaN,NaN,1
7718,P6.0,NaN,NaN,NaN,NaN,1
7719,P79.0,NaN,NaN,NaN,NaN,1
7720,P33.0,NaN,NaN,NaN,NaN,1


In [43]:
ZERO['Target'].value_counts()

Target
1    7722
Name: count, dtype: int64

In [44]:


# Step 1: Identify unique values of 'pH'
unique_size = maintenanceP['Air temperature [K]'].unique()

# Step 2: Create a dictionary to store non-missing values for each variable
# Initialize the dictionary
imputation_info = {}

# Iterate over each variable (excluding 'pH') that has missing values
for column in maintenanceP.columns:
    if column != 'Air temperature [K]' and ZERO[column].isna().sum() > 0:
        imputation_info[column] = {}
        
        # Iterate over each unique value of 'pH'
        for size_value in unique_size:
            # Get the non-missing values of the variable where 'pH' is equal to the current pH_value
            non_missing_values = maintenanceP.loc[maintenanceP['Air temperature [K]'] == size_value, column].dropna().values
            imputation_info[column][size_value] = non_missing_values

In [45]:
# import random
import statistics
def fill_missing_values2(row, imputation_info):
    # For each column, check if it has a missing value
    for column in imputation_info:
        if pd.isna(row[column]):
            size_value = row['Air temperature [K]']  # Get the corresponding pH value for the row
            if size_value in imputation_info[column]:
                possible_values = imputation_info[column][size_value]
                if len(possible_values) > 0:
                    row[column ] = statistics.mode(possible_values)
                    # row[column] = random.choice(possible_values)

    return row

myMode = ZERO.apply(lambda row: fill_missing_values2(row, imputation_info), axis=1)

In [46]:
myMode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7722 entries, 0 to 7721
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Air temperature [K]      7722 non-null   object
 1   Process temperature [K]  7722 non-null   object
 2   Rotational speed [rpm]   7722 non-null   object
 3   Torque [Nm]              7722 non-null   object
 4   Tool wear [min]          7722 non-null   object
 5   Target                   7722 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 362.1+ KB


In [47]:
# myMode[myMode.isnull().any(axis=1)]['Air temperature [K]'].value_counts()
# myMode = myMode.fillna(myMode.iloc[0])

In [48]:
print(myMode.shape)
print(ZERO.shape)

(7722, 6)
(7722, 6)


In [49]:
percentiles = [*range(0,101, 1)]
mapping_data = {}

columns_to_impute = maintenance.columns.difference(['Target'])
for variable in columns_to_impute:
    mapping_data[variable] = [maintenance[maintenance['Target']==1][variable].quantile(p / 100) for p in percentiles]

# Create the mapping DataFrame
mapping_df = pd.DataFrame(mapping_data, index=[f'P{p}' for p in percentiles])
print("Mapping DataFrame:")
mapping_df

Mapping DataFrame:


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm]
P0,295.600,306.100,1181.00,0.00,3.800
P1,296.338,307.238,1223.90,2.00,5.676
P2,296.776,307.476,1235.76,5.76,9.180
P3,296.914,307.714,1255.14,8.14,9.926
P4,297.000,307.800,1258.00,10.52,11.400
...,...,...,...,...,...
P96,303.600,312.400,2673.92,229.96,69.648
P97,303.686,312.400,2703.90,231.86,70.586
P98,303.724,312.824,2724.84,234.00,71.848
P99,303.862,313.100,2829.96,242.96,74.158


In [50]:
def convert_percentiles_to_values(mdf, mapping_df):
    columns_to_impute = mdf.columns.difference(['Target'])

    for column in columns_to_impute:
        mdf[column] = mdf[column].apply(lambda x: mapping_df.loc[x, column] if isinstance(x, str) and x.startswith('P') else x)
    return mdf

In [51]:
def path_to_revert(now, then):
    now = now.where(then.isna(), then)
    now = now.replace(r'^(P\d+)\.0$', r'\1', regex=True)
    return now

In [52]:
a = path_to_revert(myMode, ZERO)
a = convert_percentiles_to_values(a, mapping_df)

In [53]:
df_resampled2[df_resampled2['Target']==1]

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
50,298.9,309.1,2861,4.6,143,1
69,298.9,309.0,1410,65.7,191,1
77,298.8,308.9,1455,41.3,208,1
160,298.4,308.2,1282,60.7,216,1
161,298.3,308.1,1412,52.3,218,1
...,...,...,...,...,...,...
9758,298.6,309.8,2271,16.2,218,1
9764,298.5,309.5,1294,66.7,12,1
9822,298.5,309.4,1360,60.9,187,1
9830,298.3,309.3,1337,56.1,206,1


In [81]:
# a[0:339] = df_resampled2[df_resampled2['Target']==1]

In [54]:
a

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
0,298.9,309.198,2829.96,5.676,143.34,1
1,298.9,309.000,1406.80,65.706,191.00,1
2,298.8,308.900,1459.80,41.366,207.50,1
3,298.4,308.200,1284.00,60.636,216.00,1
4,298.3,308.028,1414.96,52.500,217.44,1
...,...,...,...,...,...,...
7717,297.5,308.300,2563.34,12.800,127.60,1
7718,297.3,308.300,1459.80,59.720,207.00,1
7719,302.6,310.400,1359.00,57.100,70.36,1
7720,300.3,309.900,1396.84,46.928,210.40,1


In [55]:
percent = pd.concat([a, df_resampled2[df_resampled2['Target']==0][0:7722]], ignore_index=True)

In [56]:
percent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15444 entries, 0 to 15443
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Air temperature [K]      15444 non-null  float64
 1   Process temperature [K]  15444 non-null  float64
 2   Rotational speed [rpm]   15444 non-null  float64
 3   Torque [Nm]              15444 non-null  float64
 4   Tool wear [min]          15444 non-null  float64
 5   Target                   15444 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 724.1 KB


In [57]:
def evaluate_oversampling2(X, y, classifier):

    # Train-test split on resampled data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(xtest)
    
    # Evaluate metrics
    accuracy = accuracy_score(ytest, y_pred)
    precision = precision_score(ytest, y_pred, average='weighted')
    recall = recall_score(ytest, y_pred, average='weighted')
    f1 = f1_score(ytest, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1

In [58]:
classifiers = {
    "GaussianNaiveBayes": GaussianNB(),
    "KNN": KNeighborsClassifier(),
    "RandomForest": RandomForestClassifier(),
}

Percentile_Results = []

for name, classifier in classifiers.items():
    accuracy, precision, recall, f1 = evaluate_oversampling2(
        percent[['Air temperature [K]','Process temperature [K]', 'Rotational speed [rpm]','Torque [Nm]','Tool wear [min]' ]]
        , percent[['Target']],
        classifier
    )
    Percentile_Results.append({
        "Classifier": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

Percentile_Results = pd.DataFrame(Percentile_Results)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [59]:
print('Percentile\n',Percentile_Results,'\n')
print('GaussianNaiveBayes\n',resultsNB,'\n')
print('KNN\n',resultsKNN,'\n')
print('RandomForest\n',resultsRF)

# Percentile
#             Classifier  Accuracy  Precision  Recall  F1 Score
# 0  GaussianNaiveBayes    0.9285   0.961643  0.9285  0.942561
# 1                 KNN    0.9705   0.962119  0.9705  0.963716
# 2        RandomForest    0.9795   0.979924  0.9795  0.974419 

# GaussianNaiveBayes
#                Method  Accuracy  Precision  Recall  F1 Score
# 0  RandomOverSampler    0.8470   0.963643  0.8470  0.894042
# 1              SMOTE    0.8405   0.962136  0.8405  0.889877
# 2             ADASYN    0.7920   0.965279  0.7920  0.859620
# 3    BorderlineSMOTE    0.7970   0.962743  0.7970  0.862701
# 4           SVMSMOTE    0.8725   0.960903  0.8725  0.909122 

# KNN
#                Method  Accuracy  Precision  Recall  F1 Score
# 0  RandomOverSampler    0.9300   0.956112  0.9300  0.941681
# 1              SMOTE    0.8870   0.961009  0.8870  0.917805
# 2             ADASYN    0.8820   0.961368  0.8820  0.914890
# 3    BorderlineSMOTE    0.9170   0.956139  0.9170  0.934222
# 4           SVMSMOTE    0.9235   0.956724  0.9235  0.938150 

# RandomForest
#                Method  Accuracy  Precision  Recall  F1 Score
# 0  RandomOverSampler    0.9840   0.982843  0.9840  0.983152
# 1              SMOTE    0.9575   0.972753  0.9575  0.963481
# 2             ADASYN    0.9585   0.974653  0.9585  0.964649
# 3    BorderlineSMOTE    0.9670   0.973221  0.9670  0.969610
# 4           SVMSMOTE    0.9705   0.976494  0.9705  0.972913

Percentile
            Classifier  Accuracy  Precision  Recall  F1 Score
0  GaussianNaiveBayes    0.8745   0.964756  0.8745  0.910948
1                 KNN    0.9655   0.969007  0.9655  0.967103
2        RandomForest    0.9915   0.991327  0.9915  0.991396 

GaussianNaiveBayes
               Method  Accuracy  Precision  Recall  F1 Score
0  RandomOverSampler    0.8440   0.964175  0.8440  0.892258
1              SMOTE    0.8380   0.962688  0.8380  0.888400
2             ADASYN    0.7920   0.965279  0.7920  0.859620
3    BorderlineSMOTE    0.7955   0.962703  0.7955  0.861740
4           SVMSMOTE    0.8885   0.959220  0.8885  0.918337 

KNN
               Method  Accuracy  Precision  Recall  F1 Score
0  RandomOverSampler    0.9300   0.956112  0.9300  0.941681
1              SMOTE    0.8905   0.962449  0.8905  0.920173
2             ADASYN    0.8790   0.961217  0.8790  0.913069
3    BorderlineSMOTE    0.9170   0.956774  0.9170  0.934410
4           SVMSMOTE    0.9255   0.956281  0.9255  0.93